In [1]:
# Ishan Jain
# 6 class amyloid classification

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import torch.utils.data
import torch.nn as nn
import skimage.draw
import pathlib
import collections
import tqdm
import Echo

amyloidpath = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\Amyloid\\AmyloidEcho"
sys.path.append(amyloidpath)
import echonet

verbose = False


frames = 16
period = 2
batch_size = 10
num_epochs = 40
resolution_h = 112
resolution_w = 112
lossfn = nn.CrossEntropyLoss() #nn.L1Loss()


In [2]:
# Training Classification Model 
# 6 classes
# Sweep of sizes

for i in [ 4, 5]:
    
    current_h = resolution_h * i 
    current_w = resolution_w * i 
    print(i, current_h, current_w)
    
    torch.cuda.empty_cache()

    classifications = "D:\\ExpertCohortsFullRez\\Hypertrophy-Manifest.csv"

    categories = ['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames']


    device = torch.device("cuda")

    train_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "train",
                            target_type = "Classification",
                            length = frames, 
                            period = period,                 
                            resolution_h = current_h,
                            resolution_w = current_w,                                 
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                             batch_size = 2  , 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))


    val_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "val",
                            target_type = "Classification",
                            length = frames, 
                            period = period,
                            resolution_h = current_h,
                            resolution_w = current_w,       
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size = 2 , 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))

    dataloaders = {'train':train_dataloader, 'val':val_dataloader}


    model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = 6) #r3d_18

    startingEpoch = 0
    startingBestLoss = 100000

    priorCheckpoint =  os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_checkpoint.pt")

    toGPU = torch.cuda.is_available()
    #toGPU = False

    if toGPU:
        print("cuda is available")
        device = torch.device("cuda")
        model = torch.nn.DataParallel(model)
        model.to(device)

    if os.path.exists(priorCheckpoint):   
        if toGPU:
            print("loading weights")
            checkpoint = torch.load(priorCheckpoint)
            model.load_state_dict(checkpoint['state_dict'])
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']
        else:
            print("cuda is not available, cpu weights")
            device = torch.device("cpu")
            checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
            state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
            model.load_state_dict(state_dict_cpu)
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']

    learning_rate = 1e-4
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
    bestLoss = startingBestLoss

    priorLines = None
    logFilename = os.path.join(amyloidpath, "scripts", "log_AmyloidVideoSweep"+str(current_h)+'x'+str(current_w)+".csv")


    if os.path.exists("LETS RESTART"):
        with open(logFilename, 'r') as output:
            priorLines = output.readlines()

    # with open(logFilename, 'w') as output:
    #     for line in priorLines:
    #         output.write(line)

    with open(logFilename, 'w') as output:
        for epoch in range(startingEpoch + 1, num_epochs):
            if priorLines is not None:
                for line in priorLines:
                    output.write(line)
                    print(line)

            for phase in dataloaders.keys():
                total = 0
                n = 0

                with torch.set_grad_enabled(phase == "train"):
                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                        for i, (x, outcome) in enumerate(dataloaders[phase]):
                            if verbose:
                                print(i, x.shape, outcome.shape)

                            x = x.to(device)
                            outcome = outcome.to(device)

                            p = model(x)#["out"]

                            if verbose:
                                print( x.shape, p.shape, outcome.shape)

                            loss = lossfn(p, outcome)

                            total += loss.item()
                            n += x.shape[0]

                            progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                            progressbar.update()

                            if(phase == 'train'):
                                optim.zero_grad()
                                loss.backward()
                                optim.step()

                output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")

                save = {
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'period': period,
                    'frames':frames,
                    'best_loss': bestLoss,
                    'loss': total / n,
                    'opt_dict': optim.state_dict()
                    }

                torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_checkpoint.pt"))

                if total / n < bestLoss and phase == "val":
                    torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_best.pt"))
                    bestLoss = total / n
                    print("this is the best yet")

4 448 448
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available


  0%|                                                    | 1/3651 [00:38<38:33:37, 38.03s/it, train1.0 0.9854 (0.9854)]C:\Users\Windows\Anaconda3\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|████████████████████████████████████████████████████████| 766/766 [05:00<00:00,  2.55it/s, val1.0 0.7540 (0.5476)]


this is the best yet


100%|████████████████████████████████████████████████████████| 766/766 [04:59<00:00,  2.56it/s, val2.0 0.7455 (0.5647)]


this is the best yet


...

this is the best yet


100%|███████████████████████████████████████████████████████| 766/766 [04:56<00:00,  2.59it/s, val39.0 0.6220 (0.7595)]


5 560 560
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available


  0%|                                                                                         | 0/3651 [00:37<?, ?it/s]


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\parallel\parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torchvision\models\video\resnet.py", line 230, in forward
    x = self.layer1(x)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\container.py", line 92, in forward
    input = module(input)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib

In [2]:
# Training Classification Model 
# 6 classes
# Sweep of sizes

for i in [ 3 ]:
    
    current_h = resolution_h * i 
    current_w = resolution_w * i 
    print(i, current_h, current_w)
    
    torch.cuda.empty_cache()

    classifications = "D:\\ExpertCohortsFullRez\\Hypertrophy-Manifest.csv"

    categories = ['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames']


    device = torch.device("cuda")

    train_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "train",
                            target_type = "Classification",
                            length = frames, 
                            period = period,                 
                            resolution_h = current_h,
                            resolution_w = current_w,                                 
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                             batch_size = 2  , 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))


    val_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "val",
                            target_type = "Classification",
                            length = frames, 
                            period = period,
                            resolution_h = current_h,
                            resolution_w = current_w,       
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size = 2 , 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))

    dataloaders = {'train':train_dataloader, 'val':val_dataloader}


    model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = 6) #r3d_18

    startingEpoch = 0
    startingBestLoss = 100000

    priorCheckpoint =  os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_checkpoint.pt")

    toGPU = torch.cuda.is_available()
    #toGPU = False

    if toGPU:
        print("cuda is available")
        device = torch.device("cuda")
        model = torch.nn.DataParallel(model)
        model.to(device)

    if os.path.exists(priorCheckpoint):   
        if toGPU:
            print("loading weights")
            checkpoint = torch.load(priorCheckpoint)
            model.load_state_dict(checkpoint['state_dict'])
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']
        else:
            print("cuda is not available, cpu weights")
            device = torch.device("cpu")
            checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
            state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
            model.load_state_dict(state_dict_cpu)
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']

    learning_rate = 1e-4
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
    bestLoss = startingBestLoss

    priorLines = None
    logFilename = os.path.join(amyloidpath, "scripts", "log_AmyloidVideoSweep"+str(current_h)+'x'+str(current_w)+".csv")


    if os.path.exists("LETS RESTART"):
        with open(logFilename, 'r') as output:
            priorLines = output.readlines()

    # with open(logFilename, 'w') as output:
    #     for line in priorLines:
    #         output.write(line)

    with open(logFilename, 'w') as output:
        for epoch in range(startingEpoch + 1, num_epochs):
            if priorLines is not None:
                for line in priorLines:
                    output.write(line)
                    print(line)

            for phase in dataloaders.keys():
                total = 0
                n = 0

                with torch.set_grad_enabled(phase == "train"):
                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                        for i, (x, outcome) in enumerate(dataloaders[phase]):
                            if verbose:
                                print(i, x.shape, outcome.shape)

                            x = x.to(device)
                            outcome = outcome.to(device)

                            p = model(x)#["out"]

                            if verbose:
                                print( x.shape, p.shape, outcome.shape)

                            loss = lossfn(p, outcome)

                            total += loss.item()
                            n += x.shape[0]

                            progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                            progressbar.update()

                            if(phase == 'train'):
                                optim.zero_grad()
                                loss.backward()
                                optim.step()

                output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")

                save = {
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'period': period,
                    'frames':frames,
                    'best_loss': bestLoss,
                    'loss': total / n,
                    'opt_dict': optim.state_dict()
                    }

                torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_checkpoint.pt"))

                if total / n < bestLoss and phase == "val":
                    torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"x"+str(frames)+"frames_CEloss_best.pt"))
                    bestLoss = total / n
                    print("this is the best yet")

                  

3 336 336
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available


  0%|                                                    | 1/3651 [00:36<36:52:46, 36.37s/it, train1.0 0.9353 (0.9353)]C:\Users\Windows\Anaconda3\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|████████████████████████████████████████████████████████| 766/766 [03:23<00:00,  3.76it/s, val1.0 0.7504 (0.4278)]


this is the best yet


100%|████████████████████████████████████████████████████████| 766/766 [03:21<00:00,  3.80it/s, val2.0 0.7342 (0.9144)]


this is the best yet


...

this is the best yet


100%|███████████████████████████████████████████████████████| 766/766 [03:24<00:00,  3.75it/s, val39.0 0.5425 (0.3057)]


this is the best yet
4 448 448
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available


  0%|                                                    | 1/3651 [00:36<36:41:21, 36.19s/it, train1.0 0.8537 (0.8537)]


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\parallel\parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torchvision\models\video\resnet.py", line 233, in forward
    x = self.layer4(x)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\container.py", line 92, in forward
    input = module(input)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib

In [2]:
# Training Classification Model 
# 6 classes
# Sweep of sizes

for i in [ 2 ]:
    
    current_h = resolution_h * i 
    current_w = resolution_w * i 
    print(i, current_h, current_w)
    
    torch.cuda.empty_cache()

    classifications = "D:\\ExpertCohortsFullRez\\Hypertrophy-Manifest.csv"

    categories = ['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames']


    device = torch.device("cuda")

    train_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "train",
                            target_type = "Classification",
                            length = frames, 
                            period = period,                 
                            resolution_h = current_h,
                            resolution_w = current_w,                                 
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                             batch_size = int(batch_size / i / 1.5) + 1, 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))


    val_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                            split = "val",
                            target_type = "Classification",
                            length = frames, 
                            period = period,
                            resolution_h = current_h,
                            resolution_w = current_w,       
                            classification_file = classifications,
                            num_classes = 6, 
                            categories = categories)

    val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size = int(batch_size / i / 1.5 ) + 1, 
                                             num_workers = 10,
                                             shuffle = True, 
                                             pin_memory=(device.type == "cuda"))

    dataloaders = {'train':train_dataloader, 'val':val_dataloader}


    model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = 6) #r3d_18

    startingEpoch = 0
    startingBestLoss = 100000

    priorCheckpoint =  os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"_CEloss_checkpoint.pt")

    toGPU = torch.cuda.is_available()
    #toGPU = False

    if toGPU:
        print("cuda is available")
        device = torch.device("cuda")
        model = torch.nn.DataParallel(model)
        model.to(device)

    if os.path.exists(priorCheckpoint):   
        if toGPU:
            print("loading weights")
            checkpoint = torch.load(priorCheckpoint)
            model.load_state_dict(checkpoint['state_dict'])
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']
        else:
            print("cuda is not available, cpu weights")
            device = torch.device("cpu")
            checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
            state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
            model.load_state_dict(state_dict_cpu)
            startingEpoch = checkpoint['epoch']
            startingBestLoss = checkpoint['best_loss']

    learning_rate = 1e-4
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
    bestLoss = startingBestLoss

    priorLines = None
    logFilename = os.path.join(amyloidpath, "scripts", "log_AmyloidVideoSweep"+str(current_h)+'x'+str(current_w)+".csv")


    if os.path.exists("LETS RESTART"):
        with open(logFilename, 'r') as output:
            priorLines = output.readlines()

    # with open(logFilename, 'w') as output:
    #     for line in priorLines:
    #         output.write(line)

    with open(logFilename, 'w') as output:
        for epoch in range(startingEpoch + 1, num_epochs):
            if priorLines is not None:
                for line in priorLines:
                    output.write(line)
                    print(line)

            for phase in dataloaders.keys():
                total = 0
                n = 0

                with torch.set_grad_enabled(phase == "train"):
                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                        for i, (x, outcome) in enumerate(dataloaders[phase]):
                            if verbose:
                                print(i, x.shape, outcome.shape)

                            x = x.to(device)
                            outcome = outcome.to(device)

                            p = model(x)#["out"]

                            if verbose:
                                print( x.shape, p.shape, outcome.shape)

                            loss = lossfn(p, outcome)

                            total += loss.item()
                            n += x.shape[0]

                            progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                            progressbar.update()

                            if(phase == 'train'):
                                optim.zero_grad()
                                loss.backward()
                                optim.step()

                output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")

                save = {
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'period': period,
                    'frames':frames,
                    'best_loss': bestLoss,
                    'loss': total / n,
                    'opt_dict': optim.state_dict()
                    }

                torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"_CEloss_checkpoint.pt"))

                if total / n < bestLoss and phase == "val":
                    torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(current_h)+'x'+str(current_w)+"_CEloss_best.pt"))
                    bestLoss = total / n
                    print("this is the best yet")

                  

2 224 224
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available


  0%|                                                    | 1/1826 [00:36<18:33:21, 36.60s/it, train1.0 0.4385 (0.4385)]C:\Users\Windows\Anaconda3\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|████████████████████████████████████████████████████████| 383/383 [02:30<00:00,  2.54it/s, val1.0 0.3630 (0.3304)]


this is the best yet


100%|████████████████████████████████████████████████████████| 383/383 [02:30<00:00,  2.55it/s, val2.0 0.3507 (0.3952)]


this is the best yet


...

this is the best yet


100%|███████████████████████████████████████████████████████| 383/383 [02:29<00:00,  2.55it/s, val26.0 0.2686 (0.3535)]


this is the best yet


 80%|████████████████████████████████████████▉          | 1467/1826 [40:07<09:49,  1.64s/it, train40.0 0.0865 (0.0747)]


KeyboardInterrupt: 

In [2]:
# Training Classification Model 
# 6 classes

classifications = "D:\\ExpertCohortsFullRez\\Hypertrophy-Manifest.csv"

categories = ['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames']


device = torch.device("cuda")

train_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                        split = "train",
                        target_type = "Classification",
                        length = frames, 
                        period = period,                 
                        resolution_h = resolution_h,
                        resolution_w = resolution_w,                                 
                        classification_file = classifications,
                        num_classes = 6, 
                        categories = categories)
                                
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                         batch_size = batch_size, 
                                         num_workers = 10,
                                         shuffle = True, 
                                         pin_memory=(device.type == "cuda"))


val_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                        split = "val",
                        target_type = "Classification",
                        length = frames, 
                        period = period,
                        resolution_h = resolution_h,
                        resolution_w = resolution_w,       
                        classification_file = classifications,
                        num_classes = 6, 
                        categories = categories)
                                
val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size = batch_size, 
                                         num_workers = 10,
                                         shuffle = True, 
                                         pin_memory=(device.type == "cuda"))

dataloaders = {'train':train_dataloader, 'val':val_dataloader}


model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = 6) #r3d_18

startingEpoch = 0
startingBestLoss = 100000

priorCheckpoint =  os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(resolution_h)+'x'+str(resolution_w)+"_CEloss_checkpoint.pt")

toGPU = torch.cuda.is_available()
#toGPU = False

if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)

if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
        
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss

priorLines = None
logFilename = os.path.join(amyloidpath, "scripts", "log_AmyloidVideoSweep"+str(resolution_h)+'x'+str(resolution_w)+".csv")


if os.path.exists("LETS RESTART"):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
    
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
    
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
            
        for phase in dataloaders.keys():
            total = 0
            n = 0
            
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        if verbose:
                            print(i, x.shape, outcome.shape)
                        
                        x = x.to(device)
                        outcome = outcome.to(device)
                        
                        p = model(x)#["out"]
                        
                        if verbose:
                            print( x.shape, p.shape, outcome.shape)
                        
                        loss = lossfn(p, outcome)
                        
                        total += loss.item()
                        n += x.shape[0]
                        
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
                            
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")

            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }

            torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(resolution_h)+'x'+str(resolution_w)+"_CEloss_checkpoint.pt"))

            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(resolution_h)+'x'+str(resolution_w)+"_CEloss_best.pt"))
                bestLoss = total / n
                print("this is the best yet")

                        


['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv
cuda is available
loading weights


  0%|                                                      | 1/731 [00:39<8:03:20, 39.73s/it, train2.0 0.1427 (0.1427)]C:\Users\Windows\Anaconda3\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|████████████████████████████████████████████████████████| 154/154 [01:43<00:00,  1.48it/s, val2.0 0.1367 (0.8164)]


this is the best yet


100%|████████████████████████████████████████████████████████| 154/154 [01:40<00:00,  1.53it/s, val3.0 0.1292 (0.4529)]


this is the best yet


...

100%|████████████████████████████████████████████████████████| 154/154 [01:41<00:00,  1.52it/s, val8.0 0.1194 (1.0085)]


this is the best yet


100%|████████████████████████████████████████████████████████| 154/154 [01:41<00:00,  1.52it/s, val9.0 0.1151 (0.4566)]


this is the best yet


100%|███████████████████████████████████████████████████████| 154/154 [01:41<00:00,  1.51it/s, val59.0 0.2535 (1.8040)]


In [4]:
# Test Classification Model
# Amyloid only
i = 1

output = os.path.join(amyloidpath, "scripts", "classification_amyloidvsMany_rez"+str(resolution_h * i)+'x'+str(resolution_w * i)+".csv")
#classifications = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\Amyloid\\AmyloidEcho\\scripts\\manifest_amyloidOnly.csv"

classifications = "D:\\ExpertCohortsFullRez\\Hypertrophy-Manifest.csv"

model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = 6) #r3d_18

if torch.cuda.is_available():
    print("loading weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load( os.path.join("C:\\Datasets\\checkpoints", "amyloid_rez"+str(resolution_h)+'x'+str(resolution_w)+"_CEloss_best.pt"))
    model.load_state_dict(checkpoint['state_dict'])
    

test_dataset = Echo.Echo(root = "D:\\ExpertCohortsFullRez\\CombinedForAmyloidTraining",
                        split = "test",
                        target_type = "Classification",
                        length = frames, 
                        period = period,
                        resolution_h = resolution_h * i,
                        resolution_w = resolution_w * i,       
                        classification_file = classifications,
                        num_classes = 6, 
                        categories = categories)
                                
test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                         batch_size = int(batch_size / i), 
                                         num_workers = 10,
                                         shuffle = False, 
                                         pin_memory=(device.type == "cuda"))

total = 0 
n = 0

bestguess = []
yhat = []
y = []

with torch.set_grad_enabled(False):
            with tqdm.tqdm(total = len(test_dataloader)) as progressbar:
                for (x, outcome) in test_dataloader:
                    
                    y.append(outcome.numpy())
                    x = x.to(device)
                    outcome = outcome.to(device)

                    p = model(x)#["out"]
                    yhat.append(p.to("cpu").detach().numpy())
                    m = nn.Softmax(dim = 1)
                    
                    #for i in range(x.shape[0]):
                    #    print(p[i,:].to("cpu").detach().numpy())
                    
                    thisbestguess = np.argmax(m(p).to("cpu").detach().numpy(), axis = 1)
                    #print(bestguess, outcome)
                    bestguess.append(thisbestguess)

                    loss = lossfn(p, outcome)
                    total += loss.item()
                    n += x.shape[0]

                    #print(y, yhat, bestguess)

                    progressbar.set_postfix_str("testing" +"{:.1f} {:.4f} ({:.4f})".format(1, total / n, loss.item() /  x.shape[0]))
                    progressbar.update()


loading weights
['Amyloid_Filenames', 'AS_Filenames', 'Athletes_Filenames', 'CrNativeRez', 'HCM_Filenames', 'HTN_Filenames'] D:\ExpertCohortsFullRez\Hypertrophy-Manifest.csv


  0%|                                                                                          | 0/151 [00:34<?, ?it/s]


RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\parallel\parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torchvision\models\video\resnet.py", line 230, in forward
    x = self.layer1(x)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\container.py", line 92, in forward
    input = module(input)
  File "C:\Users\Windows\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\Windows\Anaconda3\lib

In [ ]:
#output 

#print(y, yhat, bestguess)

fnames = []

with open(os.path.join(amyloidpath, "scripts", "manifest_amyloidOnly.csv"), 'r') as f:
    for line in f.readlines():
        if line.split(',')[1] == 'test':
            fnames.append(line.split(',')[0])
            
#print(len(fnames), fnames)

with open(output, "w") as g:
    g.write('file,truth,pred,amyloid,control\n')
    for i, pair in enumerate(y):
        #print(pair, yhat[i], bestguess[i])
        for ii, item in enumerate(pair):
            
            #print(fnames[i*10 + ii],item, yhat[i][ii])
            g.write(fnames[i*10 + ii] + "," + str(item) + "," + str(bestguess[i][ii]) + 
                    "," + str(yhat[i][ii][0]) + 
                    "," + str(yhat[i][ii][1]) +  "\n")
            
            
            

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

%matplotlib inline

num_classes = 2

y = []
yhat = []
bestguess = []

amyloidpath = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\Amyloid\\AmyloidEcho"
output = os.path.join(amyloidpath, "scripts", "classification_amyloidOnly_output.csv")

with open(output, 'r') as pred:
    for i, line in enumerate(pred.readlines()):
        if i > 0:
            #print(line)
            linesplit = line.strip().split(',')
            thisy = [0,0]
            thisy[int(linesplit[1])] = 1
            y.append(thisy)
            yhat.append([float(linesplit[3]), float(linesplit[4])])
            bestguess.append(int(linesplit[2]))

#print(y, yhat)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    thisy = [thisclass[i] for thisclass in y]
    thisyhat = [thisclass[i] for thisclass in yhat]
    fpr[i], tpr[i], _ = roc_curve(thisy, thisyhat)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(np.ravel(y), np.ravel(yhat))
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])





In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 2

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

#print(bestguess)
conf_matrix_y = []

with open(output, 'r') as pred:
    for i, line in enumerate(pred.readlines()):
        if i > 0:
            #print(line)
            linesplit = line.strip().split(',')
            conf_matrix_y.append(int(linesplit[1]))

conf_matrix_bestguess = bestguess
#np.concatenate(bestguess)


array = confusion_matrix(conf_matrix_y, conf_matrix_bestguess)

print(array)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = confusion_matrix(conf_matrix_y, conf_matrix_bestguess)

df_cm = pd.DataFrame(array, range(2), range(2))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()
